In [24]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
df = pd.read_csv("/content/gdrive/MyDrive/tidydata.csv")
df['label'] = df['recommended'].apply(lambda x: 0 if x== True else 1)
X = df[['review']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(df.index.values, df.label.values, test_size=0.2, random_state=40, stratify=df.label.values)
df['type'] = ['tmp']*df.shape[0]
df.loc[X_train, 'type'] = 'train'
df.loc[X_test, 'type'] = 'test'
X_train_list = list(df[df.type=='train'].review.values)
Y_train_list = list(df[df.type=='train'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_train_list)):
    if X_train_list[i]==X_train_list[i]:
        tmp1.append(X_train_list[i])
        tmp2.append(Y_train_list[i])
X_train_list = tmp1
Y_train_list = tmp2

X_test_list = list(df[df.type=='test'].review.values)
Y_test_list = list(df[df.type=='test'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_test_list)):
    if X_test_list[i]==X_test_list[i]:
        tmp1.append(X_test_list[i])
        tmp2.append(Y_test_list[i])
X_test_list = tmp1
Y_test_list = tmp2

In [26]:
import json
with open("/content/gdrive/MyDrive/df_train_w2v.json", "r") as file:
  train_emb = json.load(file)
with open("/content/gdrive/MyDrive/df_test_w2v.json", "r") as file:
  test_emb = json.load(file)

In [ ]:
train_emb[0][0]

0.10916536445419976

In [37]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(train_emb, Y_train_list)
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
#NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128, 2))
LR = LogisticRegression(random_state=0).fit(X_train, Y_train)
#NN.fit(X_train, Y_train)
LR.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=0)

In [38]:
from sklearn.metrics import classification_report
y_pred = LR.predict(test_emb)
acc = 0
for i in range(len(y_pred)):
  if y_pred[i]==Y_test_list[i]:
    acc += 1
acc = acc/len(y_pred)
print(acc)
print(classification_report(y_pred, Y_test_list, digits=3))

0.8325318586198606
              precision    recall  f1-score   support

           0      0.822     0.841     0.831     20373
           1      0.843     0.825     0.834     21217

    accuracy                          0.833     41590
   macro avg      0.833     0.833     0.833     41590
weighted avg      0.833     0.833     0.833     41590



In [31]:
sum(Y_test_list)

20748

In [46]:
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test_list, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.8157034019134333

In [40]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

GaussianNB()

In [41]:
y_pred = gnb.predict(test_emb)
acc = 0
for i in range(len(y_pred)):
  if y_pred[i]==Y_test_list[i]:
    acc += 1
acc = acc/len(y_pred)
print(classification_report(y_pred, Y_test_list, digits=3))

              precision    recall  f1-score   support

           0      0.500     0.747     0.599     13948
           1      0.830     0.623     0.712     27642

    accuracy                          0.665     41590
   macro avg      0.665     0.685     0.655     41590
weighted avg      0.719     0.665     0.674     41590



In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X_train, Y_train = shuffle(train_emb, Y_train_list)
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, Y_train)
#df_tfidf_test
print(classification_report(clf.predict(test_emb), Y_test_list, digits=3))

              precision    recall  f1-score   support

           0      0.804     0.824     0.814     20336
           1      0.827     0.808     0.817     21254

    accuracy                          0.816     41590
   macro avg      0.816     0.816     0.816     41590
weighted avg      0.816     0.816     0.816     41590



In [45]:
y_pred = clf.predict(test_emb)